In [52]:
"""
250515
1. 조회수, 업로드일, 제품 개수 크롤링 -> 완료
2. 제품 정보 가져오기(이미지, 제품명, 구매 링크, 가격) -> 완료
3. 반복 작업의 코드는 함수화 변환 -> 완료
4. 제품 가격에 있는 ₩ 표시 삭제 -> 완료
"""
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from typing import List, Tuple, Union
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import json
import time
import os
import re


# 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized') # 유튜브 더보기 클릭 및 정보 가져오기

# 크롬 드라이버 실행(자동 버전 관리)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 250515 여러 영상을 처리할 경우를 대비해서 video_ids를 리스트로 바꿈
# 드라이버가 실행될 영상 url
video_ids = ['w7uwgwj6aZM']
for video_id in video_ids:
    video_url = f'https://www.youtube.com/watch?v={video_id}'
    driver.get(video_url)

# 페이지 로딩 대기
time.sleep(5)

# 오늘 날짜 문자열 (YYYYMMDD)
today_str = datetime.today().strftime('%y%m%d')
today_str_four = datetime.today().strftime('%Y%m%d')

"""
===============
"더보기" 버튼 클릭
===============
"""
# 250515 추가
try:
    # '더보기' 버튼이 포함된 설명 섹션이 로드될 때까지 최대 20초 대기
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "description-inline-expander"))
    )
    # '더보기' 버튼이 클릭 가능 상태가 될 때까지 최대 10초 대기
    expand_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "tp-yt-paper-button#expand"))
    )
    # JavaScript를 사용하여 '더보기' 버튼 클릭 (Selenium 기본 클릭으로 안 되는 경우 대비)
    driver.execute_script("arguments[0].click();", expand_button)
    # 클릭 후 콘텐츠가 로드될 시간을 조금 기다림
    time.sleep(1)
    
except Exception as e:
    print("더보기 버튼 클릭 실패:", e)
    
"""
=====================
더보기란에 있는 텍스트 추출
=====================
"""
try:
    description = driver.find_element(By.ID, 'description-inline-expander').text
    # 설명이 비어있을 경우, 기본 메시지로 대체
    if not description.strip():
        description = "더보기란에 설명 없음"
except Exception as e:
    # 설명란 찾기 실패 또는 텍스트 추출 중 에러 발생 시
    print("더보기 클릭 또는 설명 추출 실패:", e)
    description = "더보기란에 설명 없음"
    
"""
===========================
더보기란에 있는 제품 정보 가져오기
===========================
"""
#250515 추가
# 영상 더보기에 상품 영역이 표시되는 경우
def get_product_info(driver, timeout=10):
    wait = WebDriverWait(driver, timeout)

    try:
        # 제품 정보가 담긴 요소가 페이지에 로드될 때까지 대기
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-merch-shelf-item-renderer")))
        # 상품 영역(여러 개일 수 있음) 요소들을 모두 가져오기
        product_elements = driver.find_elements(By.CSS_SELECTOR, "ytd-merch-shelf-item-renderer")

        # 최소 1개 이상 제품이 존재할 경우
        if product_elements:
            first_product = product_elements[0] # 첫 번째 제품만 사용

            # 이미지, 이름, 가격, 링크 요소 선택
            img_tag = first_product.find_element(By.CSS_SELECTOR, "#img")
            name_tag = first_product.find_element(By.CSS_SELECTOR, ".product-item-title")
            price_tag = first_product.find_element(By.CSS_SELECTOR, ".product-item-price")
            link_tag = first_product.find_element(By.CSS_SELECTOR, ".product-item-description")

            # 각 정보 추출
            product_img_url = img_tag.get_attribute("src")
            product_name = name_tag.text.strip()
            product_price = price_tag.text.replace("₩", "").strip()
            link_text = link_tag.text.strip()

            # 링크 앞에 https:// 붙이기 (이미 http로 시작하는 경우는 그대로)
            product_link = "https://" + link_text if not link_text.startswith("http") else link_text

            # 추출한 정보를 딕셔너리로 반환
            return {
                '제품 이미지 링크': product_img_url,
                '제품명': product_name,
                '제품 가격(원)': product_price,
                '제품 구매 링크': product_link
            }

    except Exception as e:
        print("제품 정보 추출 실패:", e)

    # 제품 정보를 찾지 못한 경우 기본값 반환
    return {
        '제품 이미지 링크': None,
        '제품명': None,
        '제품 가격': None,
        '제품 구매 링크': None
    }

# 함수 실행 예시
product_info = get_product_info(driver)

"""
===============================
채널명, 영상 제목, 구독자 수의 XPath
===============================
"""
# 250514 추가
# 영상에서 들고온 정보들을 담을 딕셔너리
base_info_results = {}

base_info = {
    "channel": '//*[@id="text"]/a',
    "title": '//*[@id="title"]/h1',
    "subscriber": '//*[@id="owner-sub-count"]',
}

"""
==================================================
조회수, 업로드일, 제품 수 추출 (더보기 설명란에서 정규표현식으로)
==================================================
"""
soup = BeautifulSoup(driver.page_source, 'html.parser')

#250515 추가
spans = soup.select('span.style-scope.yt-formatted-string.bold')
for span in spans:
    info_texts = span.get_text(strip=True)
    if info_texts:  # 공백 제거
        print(info_texts)

# 공백 제외하고 실제 텍스트만 추출
info_texts = [span.get_text(strip=True) for span in spans if span.get_text(strip=True)]

#250515 추가
def extract_video_info(info_texts: List[str]) -> Tuple[str, Union[str, None], Union[int, None]]:
    # 기본값 설정
    youtube_upload_date = "업로드 날짜 정보 못 찾음"
    youtube_view_count = None
    product_count = None

    # 리스트 내 문자열을 하나씩 검사
    for text in info_texts:
        # 형식: 2025. 5. 12.
        match_date_dot = re.search(r'(\d{4})\.\s*(\d{1,2})\.\s*(\d{1,2})\.', text)
        # 형식: 2025년 5월 12일
        match_date_kor = re.search(r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일', text)

        # 날짜가 dot(.) 형식인 경우
        if match_date_dot:
            year, month, day = match_date_dot.groups()
            youtube_upload_date = f"{year}{int(month):02d}{int(day):02d}"

        # 날짜가 한국어 형식인 경우
        elif match_date_kor:
            year, month, day = match_date_kor.groups()
            youtube_upload_date = f"{year}{int(month):02d}{int(day):02d}"

        # 조회수 추출: 쉼표가 포함된 숫자 (예: 472,329)
        match_views = re.search(r'조회수\s*([\d,]+)회', text)
        if match_views:
            youtube_view_count = match_views.group(1) if match_views else "조회수 정보 없음"

        # 제품 수 추출: 예) "1개 제품"
        match_products = re.search(r'(\d+)\s*개\s*제품', text)
        if match_products:
            youtube_product_count = int(match_products.group(1)) if match_products else "포함된 제품 없음"

    # 최종 결과 반환
    return youtube_upload_date, youtube_view_count, youtube_product_count

# 함수 사용 예
youtube_upload_date, youtube_view_count, youtube_product_count = extract_video_info(info_texts)

"""
=======================
구독자 수 표기 방식 변환 함수
=======================
"""
#250514 추가
def parse_subscriber_count(text):
    """
    '구독자 281만명' → '2,810,000'
    '구독자 3.45천명' → '3,450'
    '구독자 102명' → '102'
    """
    match = re.search(r'([\d\.]+)([천만]?)명', text) # 숫자+단위 추출
    if not match:
        return "0" # 매칭 안 되면 0 반환

    number = float(match.group(1)) # 숫자 부분 (정수/소수점 포함)
    unit = match.group(2) # 단위: 천/만

    # 단위에 따라 숫자 변환 
    if unit == '천':
        number *= 1_000
    elif unit == '만':
        number *= 10_000

    return f"{int(number):,}"  # 쉼표 넣은 문자열 반환

# 예: base_info = {"subscriber": ".../xpath", "channel": ".../xpath"}
for bases, xpath in base_info.items():
    try:
        # 요소가 로딩될 때까지 최대 60초 대기
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        text = element.text # 요소에서 텍스트 추출

        if bases == "subscriber":
            # 구독자 수일 경우 숫자 형식으로 변환
            base_info_results[bases] = parse_subscriber_count(text)
        else:
            # 그 외 정보는 그대로 저장
            base_info_results[bases] = text
            
    except Exception as e:
        # 오류 발생 시 메시지 출력하고 "Not found" 저장
        print(f"Error fetching {bases}: {e}")
        base_info_results[bases] = "Not found"

"""
=======================
컬럼 추가한 항목
=======================
"""
#250513 추가
#250515 조회수, 업로드일, 포함된 제품 개수의 변수명 변경, 제품 관련 정보 컬럼 추가
base_info_results.update({
    '추출일': today_str_four,
    '영상 업로드일': youtube_upload_date,
    '영상 링크': video_url,
    '조회수': youtube_view_count,
    '포함된 제품 개수': youtube_product_count,
    '더보기란 설명': description,
})

# 제품 정보도 함께 병합(제품 이미지, 제품명, 가격, 구매 링크)
base_info_results.update(product_info)

"""
=========
컬럼명 매핑
=========
"""
# 250513 추가
column_name_mapping = {
    "channel": "채널명",
    "title": "영상 제목",
    "subscriber": "구독자 수",
}
"""
=========================
원하는 컬럼 순서를 리스트로 지정
=========================
"""
# 250513 추가
column_order = [
    '추출일', '영상 업로드일', '채널명', '영상 제목', '영상 링크', '구독자 수', '조회수',
    '포함된 제품 개수', '제품 이미지 링크', '제품명', '제품 가격(원)', '제품 구매 링크', '더보기란 설명'
    ]

# 250513 추가
# DataFrame 생성 및 컬럼명 변경 반영
df = pd.DataFrame([base_info_results])
df = df.rename(columns=column_name_mapping)
# 원하는 컬럼 순서로 재정렬
df = df[column_order]

"""
==============
base info 출력
==============
"""
# 250513 추가
print("\n=== Base Info ===")
for key, value in base_info_results.items():
    print(f"{column_name_mapping.get(key, key)}: {value}")

"""
=================
파일명 중복 방지 저장
=================
"""
# 250513 추가
base_filename = f'침착맨_유튜브_{today_str}'
ext = '.xlsx'
filename = f'{base_filename}{ext}'
counter = 1
while os.path.exists(filename):
    filename = f'{base_filename}_{counter}{ext}'
    counter += 1

df.to_excel(filename, index=False)
print(f"\n✅ 엑셀 파일로 저장 완료: {filename}")

"""
==========
드라이버 종료
==========
"""
#250515 수정 (오류 처리 삭제)
driver.quit()

조회수 492,634회
2025. 5. 12.
1개 제품

=== Base Info ===
채널명: 침착맨
영상 제목: 버거킹, 우리의 아이디어를 꼭 들어주시기 바랍니다
구독자 수: 2,810,000
추출일: 20250516
영상 업로드일: 20250512
영상 링크: https://www.youtube.com/watch?v=w7uwgwj6aZM
조회수: 492,634
포함된 제품 개수: 1
더보기란 설명: 이 영상은 [버거킹]의 유료 광고가 포함되어 있습니다.

▶같이 보면 좋은 추천 영상
   • 담 걸린 침착맨의 아이언맨 맛 버거 먹방  
▶'침착맨의 일상재롱' 모아보기
   • 침착맨의 일상재롱  

▶이 영상의 생방송 원본
  •2025년 4월 28일 방송분:    • 2025년 04월 28일 1부 | 버거킹 신메뉴 크리스퍼 2종 먹방  

▶출연: 침착맨
▶편집: 15수자
▶썸네일: 15수자 

▶협업제안: info@chimchakman.com

#침착맨 #버거킹 #크리스퍼 #크리스퍼클래식 #크리스퍼클래식BLT #치킨버거 #IDONTLIKEWHOPPER #ILOVEKRISPPER #노윤서 #추영우
음악
노래 1곡
Feel The Funk
Jimmy Fontanez/Media Right Productions
Feel The Funk
음악
스크립트
스크립트를 보면서 시청하세요.
스크립트 표시
침착맨
구독자 281만명
동영상
정보
토크 영상은? 침착맨
게임 영상은? 침착맨 플러스
생방송은? 침착맨 원본 박물관
침착맨 공식 인스타그램
간략히
제품 이미지 링크: https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcTDuV-KDm4u41p9oswax6kFM54TP03poR5brwLotuD-rWE8qcZt5vBTULSF3dBuIYAdFUio0dUX
제품명: [침착맨] 빵애에요 음성 인형
제품 가격(원): 19,000
제품 구매 링크: https://playwin1.cafe24.com/shop3/product/det